# Ground Truth Data

Prepare the data

In [ ]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [ ]:
documents[0]

In [ ]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    # combined = f"{doc['course']}-{doc['question']}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [ ]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [ ]:
documents[3]

In [ ]:
from collections import defaultdict

hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [ ]:
len(hashes), len(documents)

In [ ]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [ ]:
hashes['593f7569']

In [30]:
documents

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

In [ ]:
import json

with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out)

In [31]:
prompt_template = """
You emulate a student who's taking course.
Formulate 5 questions this student might ask based on a FAQ record. 
The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record.

The record:

section = {section}
question = {question}
answer = {text}

Provide the output in parsable JSON without useing cide blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from openai import OpenAI

client = OpenAI()

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    json_response = response.choices[0].message.content
    return json_response

In [ ]:
documents[2]

In [32]:
prompt_template.format(**documents[2])

'You emulate a student who\'s taking course.\nFormulate 5 questions this student might ask based on a FAQ record. \nThe record should contain the answer to the questions, and the questions should be complete and not too short.\nIf possible, use as fewer words as possible from the record.\n\nThe record:\n\nsection = General course-related questions\nquestion = Course - Can I still join the course after the start date?\nanswer = Yes, even if you don\'t register, you\'re still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don\'t leave everything for the last minute.\n\nProvide the output in parsable JSON without useing cide blocks:\n\n["question1", "question2", ..., "question5"]'

In [33]:
generate_questions(documents[2])

'[\n  "Can I still join the course after the start date?",\n  "What happens if I don\'t register for the course?",\n  "Am I allowed to submit the homeworks without registering?",\n  "Are there deadlines for turning in the final projects?",\n  "What should I be aware of if I join the course late?"\n]'

In [34]:
from tqdm.auto import tqdm

results = {}

for doc in tqdm(documents):
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/948 [00:00<?, ?it/s]

In [35]:
results

{'c02e79ef': '[\n  "When will the course start?",\n  "How do I subscribe to the course public Google Calendar?",\n  "How do I register for the course?",\n  "How can I join the course Telegram channel?",\n  "Where can I register in DataTalks.Club\'s Slack and join the channel?"\n]',
 '1f6520ca': '[\n  "What are the prerequisites for this course?",\n  "Is there a specific textbook required for this course?",\n  "How many hours of study per week are recommended for this course?",\n  "What is the grading policy for this course?",\n  "Are there any group projects or collaborative assignments in this course?"\n]',
 '7842b56a': '[\n  "Can I still join the course after the start date?",\n  "Do I need to register to submit homeworks?",\n  "What are the deadlines for turning in the final projects?",\n  "Is there a time limit for completing the course?",\n  "Are there any prerequisites for this course?"\n]',
 '0bbf41ec': '[\n  "When can I expect to receive the confirmation email for the Data Engi

In [36]:
import pickle

with open('results.bin', 'wb') as f_in:
    pickle.dump(results, f_in)

In [38]:
results

{'c02e79ef': '[\n  "When will the course start?",\n  "How do I subscribe to the course public Google Calendar?",\n  "How do I register for the course?",\n  "How can I join the course Telegram channel?",\n  "Where can I register in DataTalks.Club\'s Slack and join the channel?"\n]',
 '1f6520ca': '[\n  "What are the prerequisites for this course?",\n  "Is there a specific textbook required for this course?",\n  "How many hours of study per week are recommended for this course?",\n  "What is the grading policy for this course?",\n  "Are there any group projects or collaborative assignments in this course?"\n]',
 '7842b56a': '[\n  "Can I still join the course after the start date?",\n  "Do I need to register to submit homeworks?",\n  "What are the deadlines for turning in the final projects?",\n  "Is there a time limit for completing the course?",\n  "Are there any prerequisites for this course?"\n]',
 '0bbf41ec': '[\n  "When can I expect to receive the confirmation email for the Data Engi

In [44]:
parsed_results = {}
failed_id = []


for doc_id, json_question in results.items():
    try:
        parsed_results[doc_id] = json.loads(json_question)
    except json.JSONDecodeError:
        print(f"Failed to parse JSON for document ID: {doc_id}")
        failed_id.append(doc_id)

Failed to parse JSON for document ID: e41b100c
Failed to parse JSON for document ID: 0b014d0c
Failed to parse JSON for document ID: eb3d6d36
Failed to parse JSON for document ID: a2c7b59f
Failed to parse JSON for document ID: c46a2e9e
Failed to parse JSON for document ID: 2d4e434f
Failed to parse JSON for document ID: 1e6b7da1
Failed to parse JSON for document ID: 18c5bafe
Failed to parse JSON for document ID: 06014eec
Failed to parse JSON for document ID: 119c917d
Failed to parse JSON for document ID: f9b673cf
Failed to parse JSON for document ID: d5b6ef5d
Failed to parse JSON for document ID: 78fce6ad
Failed to parse JSON for document ID: 27c8d5da
Failed to parse JSON for document ID: 3a98b6b7
Failed to parse JSON for document ID: 6a245a05
Failed to parse JSON for document ID: a3aa3a7d


In [51]:
print(results['e41b100c'])

1. What should I do if I receive an error translating the host name to address with Docker-Compose?
2. How can I check if the postgres database is running?
3. How can I view the containers using the command line?
4. How can I view the logs for a specific container?
5. What should I do if docker ps doesn't show pgdatabase running?


In [50]:
print(results['0b014d0c'])

[
  "How do I fix the 'undefined volume' error in Docker-Compose on Windows/WSL?",
  "What kind of error might I run into if I wrote the docker-compose.yaml file exactly like the video?",
  "What does the error 'service "pgdatabase" refers to undefined volume dtc_postgres_volume_local: invalid compose project' mean?",
  "What should I do to make it work and include the volume in my docker-compose file?",
  "Where should I add the volumes in the docker-compose file to avoid errors?"
]


In [55]:
results['eb3d6d36']

'[\n  "What is the problem with the error message \'invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe\'?",\n  "Where can I find a solution to the \'RefreshError: (\'invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe\' error?",\n]'

In [54]:
json.loads(print(results['eb3d6d36']))

[
  "What is the problem with the error message 'invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe'?",
  "Where can I find a solution to the 'RefreshError: ('invalid_grant: Invalid JWT: Token must be a short-lived token (60 minutes) and in a reasonable timeframe' error?",
]


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [48]:
json.loads(results['0b014d0c'])

JSONDecodeError: Expecting ',' delimiter: line 4 column 34 (char 221)

In [45]:
json.loads(results['e41b100c'])

JSONDecodeError: Extra data: line 1 column 2 (char 1)

In [56]:
parsed_results

{'c02e79ef': ['When will the course start?',
  'How do I subscribe to the course public Google Calendar?',
  'How do I register for the course?',
  'How can I join the course Telegram channel?',
  "Where can I register in DataTalks.Club's Slack and join the channel?"],
 '1f6520ca': ['What are the prerequisites for this course?',
  'Is there a specific textbook required for this course?',
  'How many hours of study per week are recommended for this course?',
  'What is the grading policy for this course?',
  'Are there any group projects or collaborative assignments in this course?'],
 '7842b56a': ['Can I still join the course after the start date?',
  'Do I need to register to submit homeworks?',
  'What are the deadlines for turning in the final projects?',
  'Is there a time limit for completing the course?',
  'Are there any prerequisites for this course?'],
 '0bbf41ec': ['When can I expect to receive the confirmation email for the Data Engineering Bootcamp?',
  'Is it necessary to 

In [57]:
doc_index = {d['id']: d for d in documents}
doc_index

{'c02e79ef': {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 '1f6520ca': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 '7842b56a': {'text': "Yes, even if you don't register, you're still eligibl

In [59]:
final_results = []

for doc_id, questions in parsed_results.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [60]:
import pandas as pd

df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])
df

,question,course,document
0,When will the course start?,data-engineering-zoomcamp,c02e79ef
1,How do I subscribe to the course public Google...,data-engineering-zoomcamp,c02e79ef
2,How do I register for the course?,data-engineering-zoomcamp,c02e79ef
3,How can I join the course Telegram channel?,data-engineering-zoomcamp,c02e79ef
4,Where can I register in DataTalks.Club's Slack...,data-engineering-zoomcamp,c02e79ef
...,...,...,...
4490,How can I destroy infrastructure created via G...,mlops-zoomcamp,886d1617
4491,What is the problem description for destroying...,mlops-zoomcamp,886d1617
4492,What is the solution description for destroyin...,mlops-zoomcamp,886d1617
4493,What are the commands to execute for destroyin...,mlops-zoomcamp,886d1617


In [61]:
df.to_csv('ground-truth-data.csv', index=False)